# define the DNN for mnist in ZOO attack

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys
import tempfile

from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

FLAGS = None


def deepnn(x, params):
  """deepnn builds the graph for a deep net for classifying digits.
  Args:
    x: an input tensor with the dimensions (N_examples, 784), where 784 is the
    number of pixels in a standard MNIST image.
  Returns:
    A tuple (y, keep_prob). y is a tensor of shape (N_examples, 10), with values
    equal to the logits of classifying the digit into one of 10 classes (the
    digits 0-9). keep_prob is a scalar placeholder for the probability of
    dropout.
  """
  # Reshape to use within a convolutional neural net.
  # Last dimension is for "features" - there is only one here, since images are
  # grayscale -- it would be 3 for an RGB image, 4 for RGBA, etc.
  with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1, 28, 28, 1])

  # First convolutional layer - maps one grayscale image to 32 feature maps.
  with tf.name_scope('conv1'):
    W_conv1 = weight_variable([3, 3, 1, params[0]])
    b_conv1 = bias_variable([params[0]])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
  
  with tf.name_scope('conv2'):
    W_conv2 = weight_variable([3, 3, params[0], params[1]])
    b_conv2 = bias_variable([params[1]])
    h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2) + b_conv2)
    
  # Pooling layer - downsamples by 2X.
  with tf.name_scope('pool1'):
    h_pool1 = max_pool_2x2(h_conv2)

  # Second convolutional layer -- maps 32 feature maps to 64.
  with tf.name_scope('conv3'):
    W_conv3 = weight_variable([3, 3, params[1], params[2]])
    b_conv3 = bias_variable([params[2]])
    h_conv3 = tf.nn.relu(conv2d(h_pool1, W_conv3) + b_conv3)
  
  with tf.name_scope('conv4'):
    W_conv4 = weight_variable([3, 3, params[2], params[3]])
    b_conv4 = bias_variable([params[3]])
    h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4) + b_conv4)
  # Second pooling layer.
  with tf.name_scope('pool2'):
    h_pool2 = max_pool_2x2(h_conv4)

  # Fully connected layer 1 -- after 2 round of downsampling, our 28x28 image
  # is down to 7x7x64 feature maps -- maps this to 1024 features.
  with tf.name_scope('fc1'):
    W_fc1 = weight_variable([4 * 4 * params[3], params[4]])
    b_fc1 = bias_variable([params[4]])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 4*4*params[3]])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  # Dropout - controls the complexity of the model, prevents co-adaptation of
  # features.
  with tf.name_scope('dropout'):
    keep_prob = tf.placeholder(tf.float32)
    h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
    
  with tf.name_scope('fc2'):
    W_fc2 = weight_variable([params[4], params[5]])
    b_fc2 = bias_variable([params[5]])
    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)
  
  with tf.name_scope('fc3'):
    W_fc3 = weight_variable([params[5], 10])
    b_fc3 = bias_variable([10])
    y_conv = tf.matmul(h_fc2, W_fc3) + b_fc3
    
  return y_conv, keep_prob


def conv2d(x, W):
  """conv2d returns a 2d convolution layer with full stride."""
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')


def max_pool_2x2(x):
  """max_pool_2x2 downsamples a feature map by 2X."""
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
  """weight_variable generates a weight variable of a given shape."""
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)


def bias_variable(shape):
  """bias_variable generates a bias variable of a given shape."""
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)


In [3]:
def train():
  # Import data
  mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)

  # Create the model
  x = tf.placeholder(tf.float32, [None, 784])

  # Define loss and optimizer
  y_ = tf.placeholder(tf.float32, [None, 10])

  # Build the graph for the deep net
  y_conv, keep_prob = deepnn(x,[64, 64, 128, 128, 256, 256])

  with tf.name_scope('loss'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_,
                                                            logits=y_conv)
  cross_entropy = tf.reduce_mean(cross_entropy)

  with tf.name_scope('adam_optimizer'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

  with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
  accuracy = tf.reduce_mean(correct_prediction)

  graph_location = tempfile.mkdtemp()
  print('Saving graph to: %s' % graph_location)
  train_writer = tf.summary.FileWriter(graph_location)
  train_writer.add_graph(tf.get_default_graph())

  saver = tf.train.Saver()
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(20000):
      batch = mnist.train.next_batch(50)
      if i % 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x: batch[0], y_: batch[1], keep_prob: 1.0})
        print('step %d, training accuracy %g' % (i, train_accuracy))
      train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

    saver.save(sess, '/tmp/zoo/zoocheckpoint', global_step=0)
    tf.train.write_graph(sess.graph_def, '/tmp/zoo/', 'zoo_raw_graph.pb',False)
    #can not test on all test data
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: mnist.test.images[1:50], y_: mnist.test.labels[1:50], keep_prob: 1.0}))

train()

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
Saving graph to: /tmp/tmp6ruqo2z9
step 0, training accuracy 0.08
step 100, training accuracy 0.68
step 200, training accuracy 0.82
step 300, training accuracy 0.92
step 400, training accuracy 0.98
step 500, training accuracy 0.9
step 600, training accuracy 0.92
step 700, training accuracy 0.94
step 800, training accuracy 0.9
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 0.96
step 1200, training accuracy 0.94
step 1300, training accuracy 0.98
step 1400, training accuracy 0.96
step 1500, training accuracy 0.98
step 1600, training accuracy 0.94
step 1700, training accuracy 0.96
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, trainin

# read defined model to find output node

In [5]:
sess = tf.Session()
#tf.train.write_graph(sess.graph_def, '/tmp/deepnn', 'deepnn_modelv2.pb',False)
[print(n.name) for n in tf.get_default_graph().as_graph_def().node]

Placeholder
Placeholder_1
reshape/Reshape/shape
reshape/Reshape
conv1/truncated_normal/shape
conv1/truncated_normal/mean
conv1/truncated_normal/stddev
conv1/truncated_normal/TruncatedNormal
conv1/truncated_normal/mul
conv1/truncated_normal
conv1/Variable
conv1/Variable/Assign
conv1/Variable/read
conv1/Const
conv1/Variable_1
conv1/Variable_1/Assign
conv1/Variable_1/read
conv1/Conv2D
conv1/add
conv1/Relu
conv2/truncated_normal/shape
conv2/truncated_normal/mean
conv2/truncated_normal/stddev
conv2/truncated_normal/TruncatedNormal
conv2/truncated_normal/mul
conv2/truncated_normal
conv2/Variable
conv2/Variable/Assign
conv2/Variable/read
conv2/Const
conv2/Variable_1
conv2/Variable_1/Assign
conv2/Variable_1/read
conv2/Conv2D
conv2/add
conv2/Relu
pool1/MaxPool
conv3/truncated_normal/shape
conv3/truncated_normal/mean
conv3/truncated_normal/stddev
conv3/truncated_normal/TruncatedNormal
conv3/truncated_normal/mul
conv3/truncated_normal
conv3/Variable
conv3/Variable/Assign
conv3/Variable/read
conv3

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

# Restart and load model and do inference 

In [1]:
import tensorflow as tf
import os
def create_graph():
  """Creates a graph from saved GraphDef file and returns a saver."""
  # Creates graph from saved graph_def.pb.
  with tf.gfile.FastGFile('/tmp/zoo/zoo_frozen_8bit.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    #for line in repr(graph_def).split("\n"):
    #  if "tensor_content" not in line:
    #    print(line)
    _ = tf.import_graph_def(graph_def, name='')
import numpy as np

def run_inference_on_image(image_data):
  """Runs inference on an image. (Not updated, not working for inception v3 20160828)
  Args:
    image: Image file name.
  Returns:
    Nothing
  """

  # Creates graph from saved GraphDef.
  create_graph()

  with tf.Session() as sess:
    # Some useful tensors:
    # 'softmax:0': A tensor containing the normalized prediction across
    #   1000 labels.
    # 'pool_3:0': A tensor containing the next-to-last layer containing 2048
    #   float description of the image.
    # 'DecodeJpeg/contents:0': A tensor containing a string providing JPEG
    #   encoding of the image.
    # Runs the softmax tensor by feeding the image_data as input to the graph.
    #softmax_tensor = sess.graph.get_tensor_by_name('softmax:0')
    img = tf.placeholder(tf.float32, (1,784))
    softmax_tensor = tf.import_graph_def(
            sess.graph.as_graph_def(),
            input_map={'Placeholder:0': tf.reshape(img,((1, 784))), 'dropout/Placeholder:0':1.0},
            return_elements=['fc3/add:0'])

    #dat = scipy.misc.imresize(scipy.misc.imread(image),(299,299))
    predictions = sess.run(softmax_tensor,
                           {img: image_data})

    predictions = np.squeeze(predictions)
    print(predictions)
    top_k = predictions.argsort()#[-FLAGS.num_top_predictions:][::-1]
    top_1 = top_k[-1]
    print('id',top_1)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/tensorflow/mnist/input_data', one_hot=True)
img = mnist.train.next_batch(1)
print(type(img[0][0][0]))

Extracting /tmp/tensorflow/mnist/input_data/train-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/train-labels-idx1-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-images-idx3-ubyte.gz
Extracting /tmp/tensorflow/mnist/input_data/t10k-labels-idx1-ubyte.gz
<class 'numpy.float32'>


In [3]:
img = mnist.train.next_batch(1)
print(img[1])
run_inference_on_image(img[0])

[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]]
[ -4.71347523  18.4254055   -1.71399069  -8.89132881  -2.35673761
  -2.67811108   0.74987125  -3.53510666   0.42849827  -6.10609341]
id 1
